In [0]:
!pip install pytorch_pretrained_bert
!pip install sentencepiece
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import re
import torch
from pytorch_pretrained_bert.tokenization import BertTokenizer
import os

from nltk.corpus import stopwords
import nltk
!pip install contractions
import contractions
import unicodedata
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

     |████████████████████████████████| 133kB 10.5MB/s 
     |████████████████████████████████| 1.0MB 35.0MB/s 


Using TensorFlow backend.


     |████████████████████████████████| 317kB 20.2MB/s 
     |████████████████████████████████| 245kB 45.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81693 sha256=d7bad4953e82c7824485f3759cc9cb64ad4a0c11129ed0105ec38fc8a545d29b
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [0]:
train = pd.read_csv('train.csv')
game = pd.read_csv('game_overview.csv')
test = pd.read_csv('test.csv')

In [0]:
mylist = []
for tag in game['tags']:
    mylist.extend(tag)

mylist = [x.lower().strip() for x in mylist]

uniquevalues = list(set(mylist))

In [0]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        new_word = re.sub(r'(.)\1+', r'\1\1', new_word) 
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    stop_words = []
    stop_words.append(stopwords.words('english'))
    stop_words.append(uniquevalues)
    for word in words:
        if(word not in stop_words):
            new_words.append(word)
    return new_words

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)


def removespace(text): #remove user name, special characters and urls
    # replacing the special characters and numbers
    text = re.sub(r'([^A-Za-z0-9\s])',' ',text)
    #replacing multiple spaces in to a single space
    text = re.sub(' +',' ', text)
    return text

In [0]:
def preprocess(text):
    text = removespace(text)
    text = replace_contractions(text)
    content = nltk.word_tokenize(text)
    words = remove_non_ascii(content)
    words = to_lowercase(words)
    # words = lemmatize_verbs(words)
    words = remove_punctuation(words)
    # words = remove_stopwords(words)
    # text= ' '.join(list(dict.fromkeys(words)))
    text = ' '.join(words)
    return text
def preprocess2(text):
    # text = replace_contractions(text)    
    content = nltk.word_tokenize(text)
    words = remove_non_ascii(content)
    words = to_lowercase(words)
    words = remove_stopwords(words)
    words = lemmatize_verbs(words)
    words = remove_punctuation(words)
    
    # text= ' '.join(list(dict.fromkeys(words)))
    text = ' '.join(words)
    return text

In [0]:
train['title'] = train['title'].apply(preprocess)
game['title'] = game['title'].apply(preprocess)
test['title'] = test['title'].apply(preprocess)

game['tags'] = game['tags'].apply(preprocess)
game['overview'] = game['overview'].apply(preprocess)

In [0]:
train['proc_review'] = train['user_review'].apply(preprocess)
test['proc_review'] = test['user_review'].apply(preprocess)

In [0]:
train_df = pd.merge(train, game, on='title', how='left')
test_df = pd.merge(test, game, on='title', how='left')

In [0]:
train_df.isna().sum()

review_id            0
title                0
year               178
user_review          0
user_suggestion      0
proc_review          0
developer            0
publisher            0
tags                 0
overview             0
dtype: int64

input token ids is tokenizer converts tokens using vocab file. <br>
input masks are either 0 or 1. 1 for useful tokens, 0 for padding.<br>
segment ids are either 0 or 1. For 2 text training: 0 for the first one, 1 for the second one.

MAx sequence length is choosen as 128 based on the available text length in user review

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
x_train, x_valid, y_train, y_valid = train_test_split(train_df, train_df['user_suggestion'], 
                                                      random_state=0,test_size=0.1, stratify=train_df['user_suggestion'])

## kaggle method

In [0]:
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))
 
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids
  
def create_single_input(sentence,MAX_LEN):
  
  stokens = tokenizer.tokenize(sentence)
  
  stokens = stokens[:MAX_LEN]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)

  return ids,masks,segments

def create_input_array(sentences):

  input_ids, input_masks, input_segments = [], [], []

  for sentence in tqdm(sentences,position=0, leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2)

    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [0]:
MAX_SEQ_LEN = 512
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [0]:
train_sentences = np.array(train['proc_review'])
# inputs=create_input_array(train_sentences)

In [0]:
train_sentences = np.array(train['proc_review'])
train_sentences = ["[CLS] " + sentence + " [SEP]" for sentence in train_sentences]
train_labels = train['user_suggestion'].values

In [0]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in train_sentences]

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_SEQ_LEN, dtype="long", truncating="post", padding="post")

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_SEQ_LEN, dtype="long", truncating="post", padding="post")

In [0]:
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)


In [0]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, train_labels, 
                                                            random_state=2018, test_size=0.15)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.15)

In [0]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
batch_size = 6

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
def save_checkpoint():
    checkpoint = {
        'model':model, 
    }
    torch.save(checkpoint, './bert_model.pt')
    
def load_checkpoint(filepath, inference = False):
    checkpoint = torch.load('bert_model_final.pt')
    model = checkpoint['model']
    if inference:
        for parameter in model.parameter():
            parameter.require_grad = False
        model.eval()
    model.to(device)
    return model

In [0]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# save_checkpoint()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpxkw5wj1e
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_pretrained_bert.modeling:Weights of BertForSequenceClassifi

In [0]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
for param in model.bert.parameters():
  param.requires_grad = False

for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

classifier.weight
classifier.bias


In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

# optimizer = BertAdam(optimizer_grouped_parameters,
#                      lr=2e-5,
#                      warmup=.1)

optimizer = AdamW(optimizer_grouped_parameters,
                  lr = LEARNING_RATE,
                  )

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Set our model to training mode (as opposed to evaluation mode)
model.train()

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  # Training 
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # clearing the gradient values
    optimizer.zero_grad()
    # predict the values
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,labels=None)
    loss_fct = CrossEntropyLoss(weight=class_weights)
    loss = loss_fct(logits.view(-1, num_labels), b_labels.view(-1))

    train_loss_set.append(loss.item())  
    # Perform a backward pass to calculate the gradients.
    loss.backward()
    # Clip the norm of the gradients to 1.0.
    # This is to help prevent the "exploding gradients" problem.
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    # Update the learning rate.
    scheduler.step()
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in tqdm_notebook(validation_dataloader, desc="Evaluating"):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
   
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
  save_checkpoint()

# Reference


In [0]:
train_df_bert = pd.DataFrame({
  'id':range(len(x_train)),
  'label':y_train,
  'alpha':['a']*x_train.shape[0],
  'text': x_train['proc_review'].replace(r'\n', ' ', regex=True)
})

train_df_bert.head()

,id,label,alpha,text
9078,0,1,a,planetside 2 is one of those games that you ca...
6514,1,0,a,early access reviewi will probably still play ...
10757,2,1,a,early access reviewbest digital tcg available ...
13262,3,1,a,fun little time waster while you re waiting fo...
16061,4,0,a,early access reviewsee an update dec 21st belo...


In [0]:
dev_df_bert = pd.DataFrame({
  'id':range(len(x_valid)),
  'label':y_valid,
  'alpha':['a']*x_valid.shape[0],
  'text': x_valid['proc_review'].replace(r'\n', ' ', regex=True)
})

dev_df_bert.head()

,id,label,alpha,text
2148,0,1,a,very good but beware this game is not n00bfrie...
5976,1,1,a,such an great feeling just to take off for the...
8097,2,1,a,this game is amazing after playing for a while...
13159,3,1,a,prereview warning this game contains a bug tha...
1572,4,1,a,early access review keep in mind i posted this...


In [0]:
eval_df_bert = pd.DataFrame({
  'id':range(len(test)),
  'label':[0]*test.shape[0],
  'alpha':['a']*test.shape[0],
  'text': test['proc_review'].replace(r'\n', ' ', regex=True)
})

eval_df_bert.head()

,id,label,alpha,text
0,0,0,a,nice graphics new maps weapons and models but ...
1,1,0,a,i would not recommend getting into this at its...
2,2,0,a,edit 11218i have tried playing cs go recently ...
3,3,0,a,the game is great but the community is the wor...
4,4,0,a,i thank trulyrazor for buying this for me a lo...


In [0]:
train_df_bert.to_csv('train.tsv', sep='\t', index=False, header=False)
dev_df_bert.to_csv('dev.tsv', sep='\t', index=False, header=False)
eval_df_bert.to_csv('test.tsv', sep='\t', index=False, header=False)

In [0]:
from __future__ import absolute_import, division, print_function

import csv
import os
import sys
import logging

logger = logging.getLogger()
csv.field_size_limit(2147483647) # Increase CSV reader's field limit incase we have long text.


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryClassificationProcessor(DataProcessor):
    """Processor for binary classification dataset."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

In [0]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
      total_length = len(tokens_a) + len(tokens_b)
      if total_length <= max_length:
          break
      if len(tokens_a) > len(tokens_b):
          tokens_a.pop()
      else:
          tokens_b.pop()


In [0]:
def convert_example_to_feature(example_row):
  # return example_row
  example, label_map, max_seq_length, tokenizer, output_mode = example_row

  tokens_a = tokenizer.tokenize(example.text_a)

  tokens_b = None
  if example.text_b:
      tokens_b = tokenizer.tokenize(example.text_b)
      # Modifies `tokens_a` and `tokens_b` in place so that the total
      # length is less than the specified length.
      # Account for [CLS], [SEP], [SEP] with "- 3"
      _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
      # Account for [CLS] and [SEP] with "- 2"
      if len(tokens_a) > max_seq_length - 2:
          tokens_a = tokens_a[:(max_seq_length - 2)]

  tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
  segment_ids = [0] * len(tokens)

  if tokens_b:
      tokens += tokens_b + ["[SEP]"]
      segment_ids += [1] * (len(tokens_b) + 1)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  padding = [0] * (max_seq_length - len(input_ids))
  input_ids += padding
  input_mask += padding
  segment_ids += padding

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  if output_mode == "classification":
      label_id = label_map[example.label]
  elif output_mode == "regression":
      label_id = float(example.label)
  else:
      raise KeyError(output_mode)

  return InputFeatures(input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id)

In [0]:

import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count


# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:

# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = "./"

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-cased'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'yelp'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_report/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = 'cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 512

TRAIN_BATCH_SIZE = 6
EVAL_BATCH_SIZE = 6
LEARNING_RATE = 3e-5
NUM_TRAIN_EPOCHS = 3
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [0]:
output_mode = OUTPUT_MODE

cache_dir = CACHE_DIR

In [0]:

if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)

In [0]:

if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
        raise ValueError("Output directory ({}) already exists and is not empty.".format(OUTPUT_DIR))
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [0]:
processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)

In [0]:
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)

In [0]:
num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [0]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [0]:
label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in train_examples]

In [0]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm_notebook(p.imap(convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

Preparing to convert 15744 examples..
Spawning 1 processes..


In [0]:
with open(DATA_DIR + "train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

In [0]:

# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)
# model = BertForSequenceClassification.from_pretrained(CACHE_DIR + 'cased_base_bert_pytorch.tar.gz', cache_dir=CACHE_DIR, num_labels=num_labels)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir /tmp/tmpibcaujjl
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:pytorch_pretrained_bert.modeling:Weights of BertForSequenceClassification not initialized from pretrained model: ['clas

In [0]:
import torch.nn as nn
model = nn.DataParallel(model)

In [0]:
model.to(device)

DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLa

In [0]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

In [0]:
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)

if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.float)

In [0]:

train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [0]:
processor = BinaryClassificationProcessor()
valid_examples = processor.get_dev_examples(DATA_DIR)
label_list = processor.get_labels() # [-1, 0, 1] for classification
num_labels = len(label_list)
valid_examples_len = len(valid_examples)

In [0]:
label_map = {label: i for i, label in enumerate(label_list)}
valid_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in valid_examples]

In [0]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {valid_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        valid_features = list(tqdm_notebook(p.imap(convert_example_to_feature, valid_examples_for_processing), total=valid_examples_len))

Preparing to convert 1750 examples..
Spawning 1 processes..


In [0]:
all_input_ids = torch.tensor([f.input_ids for f in valid_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in valid_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in valid_features], dtype=torch.long)

In [0]:
all_label_ids = torch.tensor([f.label_id for f in valid_features], dtype=torch.long)

In [0]:

valid_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# Run prediction for full data
valid_sampler = SequentialSampler(valid_data)
validation_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=EVAL_BATCH_SIZE)

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

In [0]:
!pip install transformers
from transformers import AdamW, BertConfig,get_linear_schedule_with_warmup


# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs


# optimizer = BertAdam(optimizer_grouped_parameters,
#                      lr=LEARNING_RATE,
#                      warmup=WARMUP_PROPORTION,
#                      t_total=num_train_optimization_steps)

optimizer = AdamW(optimizer_grouped_parameters,
                  lr = LEARNING_RATE,
                  )

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


In [0]:
weights = [0.75,1.00]
class_weights = torch.FloatTensor(weights).to(device)

In [0]:
logger.info("  Num examples = %d", train_examples_len)
logger.info("  Batch size = %d", TRAIN_BATCH_SIZE)
logger.info("  Num steps = %d", total_steps)

INFO:root:  Num examples = 15744
INFO:root:  Batch size = 6
INFO:root:  Num steps = 10496


In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [0]:
from sklearn.metrics import matthews_corrcoef, classification_report,confusion_matrix

In [0]:
def cal_eval_metrics(predictions,true_labels,epochval):
  flat_predictions = np.concatenate(predictions, axis=0)
  flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
  flat_true_labels = np.concatenate(true_labels, axis=0)
  mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
  print("Mcc and classification report for Epoch :",epochval)
  print(classification_report(flat_true_labels, flat_predictions))
  print(confusion_matrix(flat_true_labels, flat_predictions))
  print('Total MCC: %.3f' % mcc)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [0]:
import time
epochs = int(NUM_TRAIN_EPOCHS)
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    predictions , true_labels = [], []
    predictions_train , true_labels_train = [], []

    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_accuracy = 0
    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
            if step % 200 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)
            # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # As we unpack the batch, we'll also copy each tensor to the GPU using the
            batch = tuple(t.to(device) for t in batch)
        
            # Unpack this training batch from our dataloader.  
            b_input_ids, b_input_mask, b_segment_ids, b_label_ids = batch
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids
            #   [1]: segment ids 
            #   [2]: attention masks
            #   [3]: labels 
            
           
            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        
            logits = model(b_input_ids, token_type_ids=b_segment_ids, attention_mask=b_input_mask,labels=None)

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            loss_fct = CrossEntropyLoss(weight=class_weights)
            loss = loss_fct(logits.view(-1, num_labels), b_label_ids.view(-1))
            total_train_loss += loss.item()
            logits_train = logits.detach().cpu().numpy()
            label_ids_train = b_label_ids.to('cpu').numpy()
        
            predictions_train.append(logits_train)
            true_labels_train.append(label_ids_train)
        
            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_train_accuracy += flat_accuracy(logits_train, label_ids_train)
       
            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

    # Report the final accuracy for this training run.
    avg_train_accuracy = total_train_accuracy / len(train_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_train_accuracy))

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_mcc = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    
    for batch in validation_dataloader:
     
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_segment_ids, b_label_ids = batch
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = model(b_input_ids, 
                                   token_type_ids=b_segment_ids, 
                                   attention_mask=b_input_mask,
                                   labels=None)
            
        # Accumulate the validation loss.
        loss_fct = CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, num_labels), b_label_ids.view(-1))
            
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_label_ids.to('cpu').numpy()
        
        
        predictions.append(logits)
        true_labels.append(label_ids)
        
        
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    cal_eval_metrics(predictions,true_labels,epoch_i+1)
    
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 3 ========
Training...


  Batch   200  of  2,624.    Elapsed: 0:01:19.
  Batch   400  of  2,624.    Elapsed: 0:02:39.
  Batch   600  of  2,624.    Elapsed: 0:03:58.
  Batch   800  of  2,624.    Elapsed: 0:05:17.
  Batch 1,000  of  2,624.    Elapsed: 0:06:37.
  Batch 1,200  of  2,624.    Elapsed: 0:07:56.
  Batch 1,400  of  2,624.    Elapsed: 0:09:15.
  Batch 1,600  of  2,624.    Elapsed: 0:10:35.
  Batch 1,800  of  2,624.    Elapsed: 0:11:54.
  Batch 2,000  of  2,624.    Elapsed: 0:13:13.
  Batch 2,200  of  2,624.    Elapsed: 0:14:33.
  Batch 2,400  of  2,624.    Elapsed: 0:15:52.
  Batch 2,600  of  2,624.    Elapsed: 0:17:12.

  Accuracy: 0.45

  Average training loss: 0.76
  Training epcoh took: 0:17:21

Running Validation...
Mcc and classification report for Epoch : 1
              precision    recall  f1-score   support

           0       0.43      1.00      0.60       753
           1       0.00      0.00      0.00       997

    accuracy                           0.43      1750
   macro avg       0.22 

  Batch   200  of  2,624.    Elapsed: 0:01:20.
  Batch   400  of  2,624.    Elapsed: 0:02:39.
  Batch   600  of  2,624.    Elapsed: 0:03:59.
  Batch   800  of  2,624.    Elapsed: 0:05:18.
  Batch 1,000  of  2,624.    Elapsed: 0:06:37.
  Batch 1,200  of  2,624.    Elapsed: 0:07:57.
  Batch 1,400  of  2,624.    Elapsed: 0:09:16.
  Batch 1,600  of  2,624.    Elapsed: 0:10:35.
  Batch 1,800  of  2,624.    Elapsed: 0:11:54.
  Batch 2,000  of  2,624.    Elapsed: 0:13:14.
  Batch 2,200  of  2,624.    Elapsed: 0:14:33.
  Batch 2,400  of  2,624.    Elapsed: 0:15:53.
  Batch 2,600  of  2,624.    Elapsed: 0:17:12.

  Accuracy: 0.45

  Average training loss: 0.76
  Training epcoh took: 0:17:21

Running Validation...
Mcc and classification report for Epoch : 2
              precision    recall  f1-score   support

           0       0.43      1.00      0.60       753
           1       0.00      0.00      0.00       997

    accuracy                           0.43      1750
   macro avg       0.22 

  Batch   200  of  2,624.    Elapsed: 0:01:19.


KeyboardInterrupt: ignored

In [0]:
predictions

[]

# BERT TRY in pytorch

In [0]:
BERT_MODEL = 'bert-base-cased'
MAX_SEQ_LEN = 512
TRAIN_BATCH_SIZE=6
EVAL_BATCH_SIZE = 6
CACHE_DIR = './'
NUM_LABELS = 2
# num_labels=2
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 2
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1

In [0]:
from pytorch_pretrained_bert import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False)


100%|██████████| 213450/213450 [00:00<00:00, 328186.42B/s]


In [0]:
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))
 
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [0]:
def create_single_input(sentence,MAX_LEN):
  
  stokens = tokenizer.tokenize(sentence[0])
  tags = tokenizer.tokenize(sentence[1])
  
  stokens = stokens[:254]
  tags = tags[:255]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"] + tags + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)
 
  return ids,masks,segments
 
def create_input_array(sentences,tags):
 
  input_ids, input_masks, input_segments = [], [], []
 
  for sentence in tqdm_notebook(zip(sentences,tags),position=len(sentences), leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-3)
 
    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)
 
  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [0]:
x_train.columns

Index(['review_id', 'title', 'year', 'user_review', 'user_suggestion',
       'proc_review', 'developer', 'publisher', 'tags', 'overview'],
      dtype='object')

In [0]:
train_sentences = x_train['proc_review'].values
valid_sentences = x_valid['proc_review'].values
test_sentences = test['proc_review'].values

train_tags = x_train['tags'].values
valid_tags = x_valid['tags'].values
test_tags = test_df['tags'].values

train_labels = y_train.values
test_labels = [0]*test.shape[0]
valid_labels = y_valid.values

In [0]:
from tqdm import tqdm_notebook

In [0]:
train_inputs = create_input_array(train_sentences, train_tags)
valid_inputs = create_input_array(valid_sentences, valid_tags)
test_inputs = create_input_array(test_sentences, test_tags)

In [0]:
# test_inputs = create_input_array(test_sentences)

100%|██████████| 8045/8045 [00:14<00:00, 567.01it/s]


In [0]:
all_input_ids = torch.tensor([list(f) for f in train_inputs[0]], dtype=torch.long)
all_input_mask = torch.tensor([list(f) for f in train_inputs[1]], dtype=torch.long)
all_segment_ids = torch.tensor([list(f) for f in train_inputs[2]], dtype=torch.long)
all_label_ids = torch.tensor([f for f in y_train], dtype=torch.long)

In [0]:
train_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [0]:
all_input_ids = torch.tensor([list(f) for f in valid_inputs[0]], dtype=torch.long)
all_input_mask = torch.tensor([list(f) for f in valid_inputs[1]], dtype=torch.long)
all_segment_ids = torch.tensor([list(f) for f in valid_inputs[2]], dtype=torch.long)
all_label_ids = torch.tensor([f for f in y_valid], dtype=torch.long)

In [0]:
valid_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=EVAL_BATCH_SIZE)

In [0]:
all_input_ids = torch.tensor([list(f) for f in test_inputs[0]], dtype=torch.long)
all_input_mask = torch.tensor([list(f) for f in test_inputs[1]], dtype=torch.long)
all_segment_ids = torch.tensor([list(f) for f in test_inputs[2]], dtype=torch.long)
all_label_ids = torch.tensor([[f] for f in test_labels], dtype=torch.long)

In [0]:
test_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=EVAL_BATCH_SIZE)

In [0]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=NUM_LABELS)

100%|██████████| 404400730/404400730 [00:35<00:00, 11486673.13B/s]


In [0]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
!pip install transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

     |████████████████████████████████| 573kB 24.2MB/s 
     |████████████████████████████████| 3.7MB 78.5MB/s 
     |████████████████████████████████| 890kB 53.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=f6740eceb66e708b4dfd53200a0d709f81cb059eb9ff83c939034ecf0619c6db
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

In [0]:
total_steps = len(train_dataloader)*NUM_TRAIN_EPOCHS

In [0]:
num_train_optimization_steps = int(
    len(x_train) / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [0]:
# optimizer = BertAdam(optimizer_grouped_parameters,
#                      lr=LEARNING_RATE,
#                      warmup=WARMUP_PROPORTION,
#                      t_total=num_train_optimization_steps)

In [0]:
optimizer = AdamW(optimizer_grouped_parameters,
                  lr = LEARNING_RATE,
                  )
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


In [0]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

In [0]:
print("  Num examples = ", len(x_train))
print("  Batch size = ", TRAIN_BATCH_SIZE)
print("  Num steps = ", total_steps)

  Num examples =  15744
  Batch size =  6
  Num steps =  5248


In [0]:
weights = [0.75,1.00]
class_weights = torch.FloatTensor(weights).to(device)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()


In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [0]:
def cal_eval_metrics(predictions,true_labels,epochval):
  flat_predictions = np.concatenate(predictions, axis=0)
  flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
  flat_true_labels = np.concatenate(true_labels, axis=0)
  mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
  print("Mcc and classification report for Epoch :",epochval)
  print(classification_report(flat_true_labels, flat_predictions))
  print(confusion_matrix(flat_true_labels, flat_predictions))
  print('Total MCC: %.3f' % mcc)

In [0]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification

In [0]:
epochs = int(NUM_TRAIN_EPOCHS)
num_labels = 2
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    predictions , true_labels = [], []
    predictions_train , true_labels_train = [], []

    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_accuracy=0
    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_segment_ids, b_label_ids = batch
        
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        
            logits = model(b_input_ids, token_type_ids=b_segment_ids, attention_mask=b_input_mask,labels=None)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
          
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, num_labels), b_label_ids.view(-1))
            total_train_loss += loss.item()
            logits_train = logits.detach().cpu().numpy()
            label_ids_train = b_label_ids.to('cpu').numpy()
        
        
            predictions_train.append(logits_train)
            true_labels_train.append(label_ids_train)
        
        
            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_train_accuracy += flat_accuracy(logits_train, label_ids_train)
            
            
            
        # Perform a backward pass to calculate the gradients.
            loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
            optimizer.step()
            # optimizer.zero_grad()
        # Update the learning rate.
            scheduler.step()

    # Report the final accuracy for this training run.
    avg_train_accuracy = total_train_accuracy / len(train_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_train_accuracy))

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_mcc = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    
    for batch in tqdm_notebook(valid_dataloader, desc="EVALUATING"):
     
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_segment_ids, b_label_ids = batch
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = model(b_input_ids, 
                                   token_type_ids=b_segment_ids, 
                                   attention_mask=b_input_mask,
                                   labels=None)
            
        # Accumulate the validation loss.
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, num_labels), b_label_ids.view(-1))
            
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_label_ids.to('cpu').numpy()
        
        
        predictions.append(logits)
        true_labels.append(label_ids)
        
        
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    cal_eval_metrics(predictions,true_labels,epoch_i+1)
    
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(valid_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(valid_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Train Accur.' : avg_train_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )


print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 2 ========
Training...



  Accuracy: 0.97

  Average training loss: 0.11
  Training epcoh took: 0:33:16

Running Validation...



Mcc and classification report for Epoch : 1
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       753
           1       0.94      0.92      0.93       997

    accuracy                           0.92      1750
   macro avg       0.92      0.92      0.92      1750
weighted avg       0.92      0.92      0.92      1750

[[689  64]
 [ 75 922]]
Total MCC: 0.838
  Accuracy: 0.92
  Validation Loss: 0.34
  Validation took: 0:01:17

======== Epoch 2 / 2 ========
Training...



  Accuracy: 0.97

  Average training loss: 0.12
  Training epcoh took: 0:33:15

Running Validation...



Mcc and classification report for Epoch : 2
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       753
           1       0.94      0.92      0.93       997

    accuracy                           0.92      1750
   macro avg       0.92      0.92      0.92      1750
weighted avg       0.92      0.92      0.92      1750

[[689  64]
 [ 75 922]]
Total MCC: 0.838
  Accuracy: 0.92
  Validation Loss: 0.34
  Validation took: 0:01:17

Training complete!
Total training took 2:55:41 (h:mm:ss)


In [0]:
#Testing

model.eval()

# Tracking variables 
predictions_test , true_labels_test = [], []

 
for batch in tqdm_notebook(test_dataloader):
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_segment_ids, b_label_ids = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=b_segment_ids, attention_mask=b_input_mask,labels=None)

  
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_label_ids.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions_test.append(logits)
  true_labels_test.append(label_ids)

print('    DONE.')


    DONE.


In [0]:
flat_predictions = np.concatenate(predictions_test, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [0]:
test['user_suggestion'] = flat_predictions

In [0]:
test[['review_id','user_suggestion']].to_csv('sample_data/bert_submission_sh_2.csv', index=False)

In [0]:
def save_checkpoint():
  checkpoint = {'model':model}
  torch.save(checkpoint,'/content/drive/My Drive/Janatha NLP Hackathon/bert_model.pt')

In [0]:
save_checkpoint()

FileNotFoundError: ignored